<a href="https://colab.research.google.com/github/yseeker/tez_custom/blob/main/docs/seti_my_customNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!pip install --upgrade albumentations
class CFG:
    project_name = 'SETI_aug_check2'
    pretrained_model_name = 'efficientnetv2_rw_s'
    pretrained = True
    prettained_path = '../input/timm_weight/efficientnet_b0_ra-3dd342df.pth'
    mixup_alpha = 0.5
    lr = 1.0e-6
    batch_size= 16
    wandb_note = f'tawara_bs{batch_size}_adamW_default_lr{lr}_alpha{mixup_alpha}'
    input_channels = 1
    out_dim = 1
    colab_or_kaggle = 'colab'
    wandb_exp_name = f'{pretrained_model_name}_{colab_or_kaggle}_{wandb_note}'
    epochs = 60
    num_of_fold = 5
    seed = 42
    patience = 3
    delta = 0.002
    num_workers = 8
    fp16 = True
    checkpoint_path = ''
    patience_mode = 'max'
    patience = 3
    delta = 0.002
    dims_head = [None, 1]


Sat Jun 26 16:02:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import json
f = open("/content/drive/My Drive/kaggle/kaggle.json", 'r')
json_data = json.load(f) #JSON形式で読み込む
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']
os.chdir("/content/drive/My Drive/kaggle/working")

In [ ]:
os.cpu_count()

4

In [ ]:
!pip install wandb
import os
import sys
import random
#from tqdm import tqdm
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
import datetime
import psutil

import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
import torchvision
from torchsummary import summary

import cv2
from PIL import Image
import albumentations as A

import wandb
import warnings



warnings.filterwarnings("ignore")

     |████████████████████████████████| 1.8MB 8.1MB/s 
     |████████████████████████████████| 174kB 58.3MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 133kB 58.3MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=072f003e8abb3d4d183c77fec72b791f679cb8af546d78259264dffb70365a0d
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=a8ab6c20deaa221281482d72f992a8402e5872c6df9451d8c9f99f2bc885a760
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
#!pip install timm
tez_path = '../input/tez-lib'
efnet_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
timm_path = '../input/pytorch-image-models-master'
sys.path.append(tez_path)
sys.path.append(efnet_path)
sys.path.append(timm_path)

import timm


In [ ]:
timm_efficientnet_family = set(['mnasnet_050', 'mnasnet_075', 'mnasnet_100', 'mnasnet_140', 'semnasnet_050', 'semnasnet_075', 'semnasnet_100', 'semnasnet_140', 'mnasnet_small', 'mobilenetv2_100', 'mobilenetv2_110d', 'mobilenetv2_120d', 'mobilenetv2_140', 'fbnetc_100', 'spnasnet_100', 'eca_efficientnet_b0', 'gc_efficientnet_b0', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_b8', 'efficientnet_l2', 'efficientnet_es', 'efficientnet_em', 'efficientnet_el', 'efficientnet_es_pruned', 'efficientnet_el_pruned', 'efficientnet_cc_b0_4e', 'efficientnet_cc_b0_8e', 'efficientnet_cc_b1_8e', 'efficientnet_lite0', 'efficientnet_lite1', 'efficientnet_lite2', 'efficientnet_lite3', 'efficientnet_lite4', 'efficientnet_b1_pruned', 'efficientnet_b2_pruned', 'efficientnet_b3_pruned', 'efficientnetv2_rw_s', 'efficientnetv2_rw_m', 'efficientnetv2_s', 'efficientnetv2_m', 'efficientnetv2_l', 'mobilenetv3_rw'])
timm_resnet_family = set(['resnet18', 'resnet18d', 'resnet34', 'resnet34d', 'resnet26', 'resnet26d', 'resnet26t', 'resnet50', 'resnet50d', 'resnet50t', 'resnet101', 'resnet101d', 'resnet152', 'resnet152d', 'resnet200', 'resnet200d', 'tv_resnet34', 'tv_resnet50', 'tv_resnet101', 'tv_resnet152', 'wide_resnet50_2', 'wide_resnet101_2', 'resnext50_32x4d', 'resnext50d_32x4d', 'resnext101_32x4d', 'resnext101_32x8d', 'resnext101_64x4d', 'tv_resnext50_32x4d', 'ig_resnext101_32x8d', 'ig_resnext101_32x16d', 'ig_resnext101_32x32d', 'ig_resnext101_32x48d', 'ssl_resnet18', 'ssl_resnet50', 'ssl_resnext50_32x4d', 'ssl_resnext101_32x4d', 'ssl_resnext101_32x8d', 'ssl_resnext101_32x16d', 'swsl_resnet18', 'swsl_resnet50', 'swsl_resnext50_32x4d', 'swsl_resnext101_32x4d', 'swsl_resnext101_32x8d', 'swsl_resnext101_32x16d', 'seresnet18', 'seresnet34', 'seresnet50', 'seresnet50t', 'seresnet101', 'seresnet152', 'seresnet152d', 'seresnet200d', 'seresnet269d', 'seresnext26d_32x4d', 'seresnext26t_32x4d', 'seresnext50_32x4d', 'seresnext101_32x4d', 'seresnext101_32x8d', 'senet154', 'ecaresnet26t', 'ecaresnetlight', 'ecaresnet50d', 'ecaresnet50d_pruned', 'ecaresnet50t', 'ecaresnet101d', 'ecaresnet101d_pruned', 'ecaresnet200d', 'ecaresnet269d', 'ecaresnext26t_32x4d', 'ecaresnext50t_32x4d', 'resnetblur18', 'resnetblur50', 'resnetrs50', 'resnetrs101', 'resnetrs152', 'resnetrs200', 'resnetrs270', 'resnetrs350', 'resnetrs420'])
timm_vit_family = set(['vit_small_patch16_224', 'vit_base_patch16_224', 'vit_base_patch32_224', 'vit_base_patch16_384', 'vit_base_patch32_384', 'vit_large_patch16_224', 'vit_large_patch32_224', 'vit_large_patch16_384', 'vit_large_patch32_384', 'vit_base_patch16_224_in21k', 'vit_base_patch32_224_in21k', 'vit_large_patch16_224_in21k', 'vit_large_patch32_224_in21k', 'vit_huge_patch14_224_in21k', 'vit_deit_tiny_patch16_224', 'vit_deit_small_patch16_224', 'vit_deit_base_patch16_224', 'vit_deit_base_patch16_384', 'vit_deit_tiny_distilled_patch16_224', 'vit_deit_small_distilled_patch16_224', 'vit_deit_base_distilled_patch16_224', 'vit_deit_base_distilled_patch16_384', 'vit_base_patch16_224_miil_in21k', 'vit_base_patch16_224_miil'])
timm_swin_transformer = set(['swin_base_patch4_window12_384', 'swin_base_patch4_window7_224', 'swin_large_patch4_window12_384', 'swin_large_patch4_window7_224', 'swin_small_patch4_window7_224', 'swin_tiny_patch4_window7_224', 'swin_base_patch4_window12_384_in22k', 'swin_base_patch4_window7_224_in22k', 'swin_large_patch4_window12_384_in22k', 'swin_large_patch4_window7_224_in22k'])

In [ ]:
class ClassificationDataset():
    def __init__(self, image_paths, targets, transform = None): 
        self.image_paths = image_paths
        self.targets = targets
        self.transform = None

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item): 
        targets = self.targets[item]
        #image1 = np.load(self.image_paths[item]).astype(float)
        #image1 = np.load(self.image_paths[item])[::2].astype(np.float32)
        image1 = np.load(self.image_paths[item])[[0,2,4,1,3,5]]
        image = np.vstack(image1).transpose((1, 0))

        # image = ((image - np.mean(image, axis=1, keepdims=True)) / np.std(image, axis=1, keepdims=True))
        # image = ((image - np.mean(image, axis=0, keepdims=True)) / np.std(image, axis=0, keepdims=True))
    
        #image = image.astype(np.float32)[np.newaxis, ]
        image = image.astype("f")[..., np.newaxis]  # shape: (256, 819, 1)

        # image = np.load(self.image_paths[item]).astype(np.float32)
        # image = np.vstack(image).transpose((1, 0))
        # image = cv2.resize(image, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
        # image = image[np.newaxis, :, :]

        if self.transform:
            image = self.transform(image=image)["image"]

        return torch.tensor(image, dtype=torch.float), torch.tensor(targets, dtype=torch.float)

In [ ]:
def set_seed(seed = 0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:


train_aug = A.Compose(
    [
        A.Resize(p = 1.0, height = 320, width = 320),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5,
                           shift_limit = 0.2, 
                           scale_limit=0.2,
                           rotate_limit=30, 
                           border_mode = cv2.BORDER_REPLICATE),
        A.RandomResizedCrop(
            p = 1.0,
            height = 320,
            width = 320,
            scale = [0.9, 1.0]
        ),
        A.OneOf([
            A.MedianBlur(p=0.3),
            A.MotionBlur(p=0.3)
        ]
        )
    ]
)
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
df['img_path'] = df['id'].apply(
    lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy'
)
X = df.img_path.values
Y = df.target.values
skf = StratifiedKFold(n_splits = CFG.num_of_fold)

In [ ]:
def mixup_data(inputs, targets, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = inputs.size()[0]
    index = torch.randperm(batch_size)
    mixed_inputs = lam * inputs + (1 - lam) * inputs[index, :]
    targets_a, targets_b = targets, targets[index]
    
    return mixed_inputs, targets_a, targets_b, lam

def mixup_criterion(criterion, outputs, targets_a, targets_b, lam):
    return lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(CFG.pretrained_model_name, 
                                       pretrained = CFG.pretrained, 
                                       in_chans = CFG.input_channels)
        if not CFG.pretrained: self.model.load_state_dict(torch.load(CFG.pretrained_path))
        self.model.classifier = nn.Linear(self.model.classifier.in_features, CFG.out_dim)
        #self.fc = ppe.nn.LazyLinear(None, CFG.out_dim)
        self.conv1 = nn.Conv2d(1, 3, 
                               kernel_size=3, 
                               stride=1, 
                               padding=3, 
                               bias=False)
        
        dims_head = CFG.dims_head
        if dims_head[0] is None:
            dims_head[0] = self.model.classifier.in_features
        layers_list = []
        for i in range(len(dims_head) - 2):
            in_dim, out_dim = dims_head[i: i + 2]
            layers_list.extend([
                nn.Linear(in_dim, out_dim),
                nn.ReLU(), nn.Dropout(0.5),])
        layers_list.append(
            nn.Linear(dims_head[-2], dims_head[-1]))
        self.head_cls = nn.Sequential(*layers_list)

    def forward(self, inputs):
        #x = self.conv1(inputs)
        x = self.model(inputs)
        outputs = self.head_cls(x)
        return outputs

class Trainer():
    def __init__(
        self,
        model,
        train_dataset,
        valid_dataset,
        train_batchsize,
        valid_batchsize,
        valid_targets,
        num_workers,
        fp16,
        multiple_GPU,
        determinstic,
        benchmark,
    ):
        self.model = model
        self.valid_targets = valid_targets
        self.fp16 = fp16
        self.scaler = torch.cuda.amp.GradScaler()
        self.optimizer = self.configure_optimizer()
        self.scheduler_after_step = self.configure_scheduler_after_step()
        self.scheduler_after_epoch = self.configure_scheduler_after_epoch()
        torch.backends.cudnn.deterministic = determinstic
        torch.backends.cudnn.benchmark = benchmark 
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        if num_workers == -1: num_workers = psutil.cpu_count()
        self.multiple_GPU = multiple_GPU
        if multiple_GPU and torch.cuda.device_count() > 1:
            print("Let's use", torch.cuda.device_count(), "GPUs!")
            self = nn.DataParallel(self)
        self.model.to(self.device)

        self.train_loader = torch.utils.data.DataLoader(
            dataset = train_dataset, 
            batch_size = train_batchsize,
            shuffle=True, 
            num_workers= num_workers,
            drop_last = True,
            pin_memory = True
        )
        self.valid_loader = torch.utils.data.DataLoader(
            dataset = valid_dataset, 
            batch_size=valid_batchsize,
            shuffle=False, 
            num_workers = num_workers,
            drop_last = False,
            pin_memory = True
        )

    def _init_wandb(self, cfg):
        hyperparams = {
            'batch_size' : cfg.batch_size,
            'epochs' : cfg.epochs
        }
        wandb.init(
            config = hyperparams,
            project= cfg.project_name,
            name=cfg.wandb_exp_name,
        )
        wandb.watch(self.model)

    def configure_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=CFG.lr)
        #opt = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        return opt
    
    def configure_scheduler_after_step(self):
        sch = torch.optim.lr_scheduler.OneCycleLR(
            optimizer = self.optimizer,
            epochs = CFG.epochs,
            steps_per_epoch = 2508,
            max_lr = 5.0e-4,
            pct_start = 0.1,
            anneal_strategy = 'cos',
            div_factor = 1.0e+3,
            final_div_factor = 1.0e+3
        )
        # sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        #     self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        # )
        #sch = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=7, gamma=0.1)
        #sch = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=5e-4, gamma=0.9, cycle_momentum=False,
        #step_size_up=1400,step_size_down=1400, mode="triangular2")
        return sch

    def configure_scheduler_after_epoch(self):
        return None

    def epoch_metrics(self, outputs, targets):
        return metrics.roc_auc_score(targets, outputs)

    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        if len(np.unique(targets)) > 1: 
            roc_auc = metrics.roc_auc_score(targets, outputs)
        else: roc_auc = 1.0
        return roc_auc

    def train_one_step(self, inputs, targets):
        inputs = inputs.to(self.device, non_blocking=True)
        targets = targets.to(self.device, non_blocking=True)
        inputs, targets_a, targets_b, lam = mixup_data(inputs, 
                                                      targets,
                                                      alpha= CFG.mixup_alpha)
        self.optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            if self.fp16:
                with torch.cuda.amp.autocast(self.fp16):
                    outputs = self.model(inputs)
                    #loss = self.criterion(outputs, targets.view(-1, 1))
                    loss = mixup_criterion(self.criterion,
                                                outputs, 
                                                targets_a.view(-1, 1),
                                                targets_b.view(-1, 1), 
                                                lam)
                    metrics = self.monitor_metrics(outputs, targets)
                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                outputs = self(inputs)
                metrics = self.monitor_metrics(outputs, targets)
                loss = self.criterion(outputs, targets.view(-1, 1))
                loss.backward()
                self.optimizer.step()
            if self.scheduler_after_step:
                self.scheduler_after_step.step()
        return outputs, loss, metrics

    def validate_one_step(self, inputs, targets = None):
        inputs = inputs.to(self.device, non_blocking=True)
        if targets is not None:
            targets = targets.to(self.device, non_blocking=True)
            with torch.no_grad():
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets.view(-1, 1))
                metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        else:
            outputs = self(inputs)
            return outputs, None, None

    def predict_one_step(self, inputs):
        outputs, _, _ = self.validate_one_step(inputs)
        return outputs
        
    def train_one_epoch(self, data_loader):
        self.model.train()
        running_loss, running_metrics = AverageMeter(), AverageMeter()
        tk0 = tqdm(data_loader, total=len(data_loader), position = 0, leave = True)
        for batch_idx, (inputs, targets) in enumerate(tk0):
            preds_one_batch, loss, metrics = self.train_one_step(inputs, targets)
            running_loss.update(loss.item(), data_loader.batch_size)
            running_metrics.update(metrics, data_loader.batch_size)
            current_lr = self.optimizer.param_groups[0]['lr'] 
            wandb.log({
                "train/step" : b_idx,
                "train/loss_step": losses.avg,
                "lr": current_lr 
                }
            tk0.set_postfix(train_loss=running_loss.avg, train_step_metrics = running_metrics.avg, stage="train", lr = current_lr)
        if self.scheduler_after_epoch:
            self.scheduler_after_epoch.step()
        tk0.close()
        return running_loss.avg

    def validate_one_epoch(self, data_loader):
        self.model.eval()
        running_loss, running_metrics = AverageMeter(), AverageMeter()
        preds_list = []
        tk0 = tqdm(data_loader, total=len(data_loader), position = 0, leave = True)
        for batch_idx, (inputs, targets) in enumerate(tk0):
            preds_one_batch, loss, metrics = self.validate_one_step(inputs, targets)
            preds_list.append(preds_one_batch.cpu().detach().numpy())
            running_loss.update(loss.item(), data_loader.batch_size)
            running_metrics.update(metrics, data_loader.batch_size)
            tk0.set_postfix(valid_loss = running_loss.avg, validate_step_metrics = running_metrics.avg, stage="validation")
            wandb.log({
                "valid/step" : batch_idx,
                "valid/metric_step" : running_metrics.avg,
                "valid/loss": running_loss.avg, 
                })
        preds_arr = np.concatenate(preds_list)
        valid_metric_val = self.epoch_metrics(preds_arr, self.valid_targets)
        tk0.close()
        return valid_metric_val, running_loss.avg

    def predict(
        self,
        dataset,
        batch_size = 16,
        num_workers = 8,
    ):
        self.model.eval()
        self.test_loader =  torch.utils.data.DataLoader(
            dataset = test_dataset, 
            batch_size = batch_size,
            shuffle = False, 
            num_workers= num_workers,
            drop_last = False,
            pin_memory = True
        )
        preds_list = []
        tk0 = tqdm(data_loader, total=len(self.test_loader), position = 0, leave = True)
        for batch_idx, (inputs, targets) in enumerate(tk0):
            preds_one_batch = self.predict_one_step(inputs, targets)
            preds_list.append(preds_one_batch.cpu().detach().numpy())
            tk0.set_postfix(stage="inference")
        tk0.close()
        preds_arr = np.concatenate(preds_list)
        return preds_arr

    def save(self, model_path):
        model_state_dict = self.state_dict()
        if self.optimizer is not None:
            opt_state_dict = self.optimizer.state_dict()
        else:
            opt_state_dict = None
        if self.scheduler_after_step is not None:
            sch_state_dict_after_step = self.scheduler_after_step.state_dict()
        else:
            sch_state_dict_after_step = None
        if self.scheduler_after_epoch is not None:
            sch_state_dict_after_epoch = self.scheduler_after_epoch.state_dict()
        else:
            sch_state_dict_after_epoch = None
        model_dict = {}
        model_dict["state_dict"] = model_state_dict
        model_dict["optimizer"] = opt_state_dict
        model_dict["scheduler_after_step"] = sch_state_dict_after_step
        model_dict["scheduler_after_epoch"] = sch_state_dict_after_epoch
        model_dict["epoch"] = self.current_epoch
        model_dict["fp16"] = self.fp16
        model_dict["multiple_GPU"] = self.multiple_GPU
        torch.save(model_dict, model_path)

    def load(self, model_path):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        if next(self.parameters()).device != self.device:
            self.to(self.device)
        model_dict = torch.load(model_path, map_location=torch.device(device))
        self.load_state_dict(model_dict["state_dict"])

    def fit(
        self,
        cfg,
        checkpoint_save_path = '',
        mode = 'max',
        patience = 10,
        delta = 0.001,
    ):
        set_seed(CFG.seed)
        self._init_wandb(cfg)

        if mode == 'max':
            current_best_valid_metrics = -float('inf')
        else:
            current_best_valid_metrics= float('inf')
        early_stopping_counter = 0

        for epoch in range(epochs):
            train_loss = self.train_one_epoch(self.train_loader)
            if valid_dataset:
                valid_metrics, valid_loss = self.validate_one_epoch(self.valid_loader)
                # Early Stopping and save at the check points.
                if mode == 'max':
                    if valid_metrics < current_best_valid_metrics + delta:
                        early_stopping_counter += 1
                        print(f'EarlyStopping counter: {early_stopping_counter} out of {patience}')
                        if early_stopping_counter >= patience: break
                    else:
                        print(f"Validation score improved ({current_best_valid_metrics} --> {valid_metrics}). Saving the check point!")
                        current_best_validmetrics= valid_metrics
                        self.save(checkpoint_save_path + f"{cfg.pretrained_model_name}_epoch{epoch}.cpt" )
                else:
                    if valid_metrics > current_best_valid_metrics - delta:
                        early_stopping_counter += 1
                        print(f'EarlyStopping counter: {early_stopping_counter} out of {patience}')
                        if early_stopping_counter >= patience: break
                    else:
                        print(f"Validation score improved ({current_best_valid_metrics} --> {valid_metrics}). Saving the check point!")
                        current_best_valid_metrics = valid_metrics
                        self.save(checkpoint_save_path + f"{cfg.pretrained_model_name}_epoch{epoch}.cpt" )
                        
            #writer.add_scalar("Loss/train", 1.0, epoch)
            print(f'epoch: {epoch}, validate_epoch_metrics : {valid_metrics}')
            wandb.log({
                "epoch" : epoch,
                "train/loss" : train_loss,
                "valid/loss" : valid_loss,
                "valid/metric" : valid_metrics,
                })
        wandb.finish()
        torch.cuda.empty_cache()
        gc.collect()

In [ ]:
df3 = pd.read_csv('../input/seti-breakthrough-listen/valid_labels.csv')
df3['img_path'] = df3['id'].apply(
    lambda x: f'../input/seti-breakthrough-listen/valid/{x[0]}/{x}.npy'
)

X_valid = df3.img_path
Y_valid = df3.target

for fold_cnt, (train_index, test_index) in enumerate(skf.split(X, Y), 1):
    train_images, valid_images = X[train_index], X_valid
    train_targets, valid_targets = Y[train_index], Y_valid

    train_dataset = ClassificationDataset(
        image_paths=train_images, 
        targets=train_targets, 
        transform = train_aug
    )
    valid_dataset = ClassificationDataset(
        image_paths=valid_images, 
        targets=valid_targets, 
        transform = None
    )
    model = BasicNN()

    model.fit(
        cfg = CFG,
        train_dataset = train_dataset,
        valid_dataset = valid_dataset,
        valid_targets = valid_targets,
        epochs = CFG.epochs,
        train_batchsize = CFG.batch_size,
        valid_batchsize = CFG.batch_size,
        num_workers = CFG.num_workers,
        fp16 = CFG.fp16,
        checkpoint_save_path = CFG.checkpoint_path,
        mode = CFG.patience_mode,
        patience = CFG.patience,
        delta = CFG.delta
    )


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_v2s_ra2_288-a6477665.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2s_ra2_288-a6477665.pth


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


RuntimeError: ignored

In [ ]:
submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path'] = submission['id'].apply(
    lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy'
)
test_dataset = ClassificationDataset(
    image_paths=submission.img_path.values, 
    targets=submission.target.values, 
    transform = train_aug
)

final_preds = None
num_of_ave = 5
outs = []
sig = torch.nn.Sigmoid()
for model in models:
    for j in range(num_of_ave):
        preds = model.predict(test_dataset, batch_size=16, n_jobs=-1)
        temp_preds = None
        for p in preds:
            if temp_preds is None:
                temp_preds = p
            else:
                temp_preds = np.vstack((temp_preds, p))
        if final_preds is None:
            final_preds = temp_preds
        else:
            final_preds += temp_preds
    final_preds = final_preds/num_of_ave
    out = sig(torch.from_numpy(final_preds))
    out = out.detach().numpy()
    outs.append(out)
pred = np.mean(np.array(outs), axis=0)
submission.target = pred
submission.drop(['img_path'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)

In [ ]:
class ClassificationDataset():
    def __init__(self, image_paths, targets, transform = None): 
        self.image_paths = image_paths
        self.targets = targets
        self.transform = None

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item): 
        targets = self.targets[item]
        #image1 = np.load(self.image_paths[item]).astype(float)
        image1 = np.load(self.image_paths[item])[::2].astype(np.float32)
        image = np.vstack(image1).transpose((1, 0))

        image = ((image - np.mean(image, axis=1, keepdims=True)) / np.std(image, axis=1, keepdims=True))
        image = ((image - np.mean(image, axis=0, keepdims=True)) / np.std(image, axis=0, keepdims=True))
    
        image = image.astype(np.float32)[np.newaxis, ]

        # image = np.load(self.image_paths[item]).astype(np.float32)
        # image = np.vstack(image).transpose((1, 0))
        # image = cv2.resize(image, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
        # image = image[np.newaxis, :, :]

        if self.transform:
            image = self.transform(image=image)["image"]

        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }


class ClassificationDataset2():
    def __init__(self, image_paths, targets, transform = None): 
        self.image_paths = image_paths
        self.targets = targets
        self.transform = None

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item): 
        targets = self.targets[item]
        #image1 = np.load(self.image_paths[item]).astype(float)
        image1 = np.load(self.image_paths[item])[::2].astype(np.float32)
        image = np.vstack(image1).transpose((1, 0))

        image = ((image - np.mean(image, axis=1, keepdims=True)) / np.std(image, axis=1, keepdims=True))
        image = ((image - np.mean(image, axis=0, keepdims=True)) / np.std(image, axis=0, keepdims=True))
    
        image = image.astype(np.float32)[np.newaxis, ]

        # image = np.load(self.image_paths[item]).astype(np.float32)
        # image = np.vstack(image).transpose((1, 0))
        # image = cv2.resize(image, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
        # image = image[np.newaxis, :, :]

        if self.transform:
            image = self.transform(image=image)["image"]

        return torch.tensor(image, dtype=torch.float), torch.tensor(targets, dtype=torch.float)

df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
df['img_path'] = df['id'].apply(
    lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy'
)
X = df.img_path.values
Y = df.target.values



skf = StratifiedKFold(n_splits = 5)

patience = 3
delta = 0.002

for fold_cnt, (train_index, test_index) in enumerate(skf.split(X, Y), 1):
    train_images, valid_images = X[train_index], X[test_index]
    train_targets, valid_targets = Y[train_index], Y[test_index]



    av = 0
    for i in range(5):
        train_dataset1 = ClassificationDataset(
        image_paths=train_images, 
        targets=train_targets, 
        transform = None
    )
        train_loader1 = torch.utils.data.DataLoader(
                        dataset = train_dataset1, 
                        batch_size = 32,
                        shuffle=True, 
                        num_workers= 8,
                        pin_memory = True
                    )
        d = datetime.datetime.now()
        for i, data in enumerate(train_loader1):
            if i == 0: d = datetime.datetime.now()
            #print(i, datetime.datetime.now()-d)
            if i == 100: break
            pass
        t = datetime.datetime.now() - d
        #print(t)
        av += t.seconds
    print('tez-av: ', av/5, 'num_workers : ', 8)


